### Ex1 : Initialisation de l’environnement et des structures de données

In [65]:
import gymnasium as gym
import numpy as np

In [67]:
env = gym.make("Taxi-v3")

state_size = env.observation_space.n
action_size = env.action_space.n

print(f"nombre d'états : {state_size}")
print(f"nombre d'actions : {action_size}")

nombre d'états : 500
nombre d'actions : 6


In [69]:
# Table de politique 
policy_table = np.ones((state_size, action_size)) / action_size

# Table de valeurs 
value_table = np.zeros(state_size)

print("Les dix premières lignes de policy_table : \n")
print(policy_table[:10])

print("\n Les dix premières lignes de value_table :")
print(value_table[:10])

Les dix premières lignes de policy_table : 

[[0.16666667 0.16666667 0.16666667 0.16666667 0.16666667 0.16666667]
 [0.16666667 0.16666667 0.16666667 0.16666667 0.16666667 0.16666667]
 [0.16666667 0.16666667 0.16666667 0.16666667 0.16666667 0.16666667]
 [0.16666667 0.16666667 0.16666667 0.16666667 0.16666667 0.16666667]
 [0.16666667 0.16666667 0.16666667 0.16666667 0.16666667 0.16666667]
 [0.16666667 0.16666667 0.16666667 0.16666667 0.16666667 0.16666667]
 [0.16666667 0.16666667 0.16666667 0.16666667 0.16666667 0.16666667]
 [0.16666667 0.16666667 0.16666667 0.16666667 0.16666667 0.16666667]
 [0.16666667 0.16666667 0.16666667 0.16666667 0.16666667 0.16666667]
 [0.16666667 0.16666667 0.16666667 0.16666667 0.16666667 0.16666667]]

 Les dix premières lignes de value_table :
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]


## Ex2 : Exploration et collecte d’épisodes

In [82]:
for episode in range(20):
    state, _ = env.reset()
    done = False
    total_reward = 0
    
    while not done:
        action = env.action_space.sample()  
        next_state, reward, terminated, truncated, _ = env.step(action)
        
        total_reward += reward
        state = next_state
        done = terminated or truncated
    
    print(f"Épisode {episode + 1}: Action = {action}, Récompense totale = {total_reward}")
print(f"\n Récompense moyenne avant entraînement: {np.mean(total_reward)}")


Épisode 1: Action = 4, Récompense totale = -731
Épisode 2: Action = 2, Récompense totale = -776
Épisode 3: Action = 0, Récompense totale = -821
Épisode 4: Action = 3, Récompense totale = -740
Épisode 5: Action = 0, Récompense totale = -776
Épisode 6: Action = 4, Récompense totale = -812
Épisode 7: Action = 2, Récompense totale = -749
Épisode 8: Action = 3, Récompense totale = -785
Épisode 9: Action = 4, Récompense totale = -668
Épisode 10: Action = 4, Récompense totale = -830
Épisode 11: Action = 2, Récompense totale = -839
Épisode 12: Action = 5, Récompense totale = -740
Épisode 13: Action = 2, Récompense totale = -839
Épisode 14: Action = 1, Récompense totale = -794
Épisode 15: Action = 4, Récompense totale = -767
Épisode 16: Action = 2, Récompense totale = -749
Épisode 17: Action = 4, Récompense totale = -839
Épisode 18: Action = 3, Récompense totale = -758
Épisode 19: Action = 2, Récompense totale = -740
Épisode 20: Action = 3, Récompense totale = -722

 Récompense moyenne avant en

## Ex3 : Mise à jour de la politique avec PPO

In [75]:
gamma = 0.99  
lr_policy = 0.1  
clip_epsilon = 0.2  

episode_states = []
episode_actions = []
episode_rewards = []

for episode in range(20):  
    state, _ = env.reset()
    done = False
    states, actions, rewards = [], [], []
    
    while not done:
        policy_table = np.ones((state_size, action_size)) / action_size
  # Action basée sur la politique actuelle
        next_state, reward, terminated, truncated, _ = env.step(action)
        
        states.append(state)
        actions.append(action)
        rewards.append(reward)

        state = next_state
        done = terminated or truncated

    # Ajouter les données de cet épisode à l’historique global
    episode_states.append(states)
    episode_actions.append(actions)
    episode_rewards.append(rewards)

    # Calcul des récompenses cumulées
    R = 0
    discounted_rewards = []
    for r in reversed(rewards):
        R = r + gamma * R
        discounted_rewards.insert(0, R)

    # MAJ de la fonction de valeur et de la politique
    for i in range(len(states)):
        advantage = discounted_rewards[i] - value_table[states[i]]
        
        # MAJ de la politique avec clipping
        old_prob = policy_table[states[i], actions[i]]
        new_prob = old_prob + lr_policy * advantage
        policy_table[states[i], actions[i]] = np.clip(new_prob, 1 - clip_epsilon, 1 + clip_epsilon)

        # MAJ de la table de valeurs
        value_table[states[i]] += lr_policy * advantage


## Ex4 : Évaluation de l’agent après entraînement

In [78]:
num_eval_episodes = 20
total_rewards = []

for ep in range(num_eval_episodes):
    state, _ = env.reset()
    total_reward = 0
    done = False

    while not done:
        action = np.argmax(policy_table[state])  
        next_state, reward, terminated, truncated, _ = env.step(action)
        
        total_reward += reward
        state = next_state
        done = terminated or truncated

    total_rewards.append(total_reward)

print(f"Récompense moyenne après entraînement: {np.mean(total_rewards)}")


Récompense moyenne après entraînement: -200.0


Comparaison des performances : 

Avant l'entraînement : l'Agent choisit des actions aléatoirement ==> il a une faible récompense moyenne.

Après l'entraînement : l'Agent suit la politique optimisée ==> il obtieent des récompenses plus élevées.